In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [5]:
dataset = os.listdir('Downloads/dataset')

In [6]:
page = pd.read_csv('Downloads/dataset/TBN_CUST_BEHAVIOR.csv')
txn = pd.read_csv('Downloads/dataset/TBN_WM_TXN.csv')
dt = pd.read_csv('Downloads/dataset/TBN_RECENT_DT.csv')
loan = pd.read_csv('Downloads/dataset/TBN_LN_APPLY.csv')
fx = pd.read_csv('Downloads/dataset/TBN_FX_TXN.csv')
card = pd.read_csv('Downloads/dataset/TBN_CC_APPLY.csv')

In [7]:
txn.rename(columns = {'TXN_DT': 'I_TXN_DT', }, inplace = True)
card.rename(columns = {'TXN_DT': 'C_TXN_DT', }, inplace = True)
fx.rename(columns = {'TXN_DT': 'F_TXN_DT', }, inplace = True)
loan.rename(columns = {'TXN_DT': 'L_TXN_DT', }, inplace = True)

In [5]:
df_dataset = {'網頁瀏覽':page, '信託交易':txn, '基本屬性':identity, '信用卡':card, 
              '外匯交易':fx, '信貸申請':loan, '最後交易':dt}

In [6]:
for i in df_dataset:
    print(i, df_dataset[i].shape, 'unique_customer = {}'.format(len(df_dataset[i]['CUST_NO'].drop_duplicates())), 
          '\n', list(df_dataset[i].columns), end='\n\n')

網頁瀏覽 (2209864, 3) unique_customer = 195000 
 ['CUST_NO', 'VISITDATE', 'PAGE']

信託交易 (36763, 5) unique_customer = 14004 
 ['CUST_NO', 'I_TXN_DT', 'CUST_RISK_CODE', 'INVEST_TYPE_CODE', 'WM_TXN_AMT']

基本屬性 (187679, 8) unique_customer = 187679 
 ['CUST_NO', 'AGE', 'CHILDREN_CNT', 'CUST_START_DT', 'EDU_CODE', 'GENDER_CODE', 'INCOME_RANGE_CODE', 'WORK_MTHS']

信用卡 (54393, 2) unique_customer = 44112 
 ['CUST_NO', 'C_TXN_DT']

外匯交易 (507185, 3) unique_customer = 74111 
 ['CUST_NO', 'F_TXN_DT', 'FX_TXN_AMT']

信貸申請 (6741, 4) unique_customer = 6654 
 ['CUST_NO', 'L_TXN_DT', 'LN_AMT', 'LN_USE']

最後交易 (195000, 5) unique_customer = 195000 
 ['CUST_NO', 'CC_RECENT_DT', 'FX_RECENT_DT', 'LN_RECENT_DT', 'WM_RECENT_DT']



In [8]:
# 編輯資料
txn['NEW_TXN_DT'] = txn['I_TXN_DT'].apply(lambda x:x-9447)
card['NEW_TXN_DT'] = card['C_TXN_DT'].apply(lambda x:x-9447)
fx['NEW_TXN_DT'] = fx['F_TXN_DT'].apply(lambda x:x-9447)
loan['NEW_TXN_DT'] = loan['L_TXN_DT'].apply(lambda x:x-9447)

# CUST_BEHAVIOR_PATH

In [13]:
data = pd.read_csv('Downloads/TBN_CUST_BEHAVIOR_PATH.csv')

In [14]:
# 對瀏覽行為加工
page['NEW_VISITDATE'] = page['VISITDATE'].apply(lambda x:x-9447)
def add(x):
    path1 = x[0]
    path2 = x[1]
    return '/'.join([x[0],x[1]])
data['new_path'] = data[['PATH1', 'PATH2']].apply(add, axis=1)
d_page = data[['CUST_NO', 'VISITDATE', 'new_path']]
d_page['NEW_VISITDATE'] = d_page['VISITDATE'].apply(lambda x:x-9447)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
d_page.head()

,CUST_NO,VISITDATE,new_path,NEW_VISITDATE
0,---CHVW7DUN8SZLO,9472,edrn/pgusordq,25
1,---CHVW7DUN8SZLO,9472,gygrt/cugfkt,25
2,---CHVW7DUN8SZLO,9499,edrn/deoxt,52
3,---CHVW7DUN8SZLO,9499,edrn/pgusordq,52
4,---CHVW7DUN8SZLO,9499,edrn/pgusordq,52


# identity

In [692]:
identity = pd.read_csv('Downloads/dataset/TBN_CIF.csv')
identity.head()

,CUST_NO,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,UFUIMEGMK2KTIWI4,1,NaN,2.0,NaN,NaN,NaN,NaN
1,3HJN2X_LGBHBKILI,3,0.0,9393.0,NaN,NaN,1.0,NaN
2,MNEZK2O3OE-JHYBW,4,0.0,2912.0,3.0,M,2.0,2.0
3,W7ADZSWYXEMHB7JQ,4,0.0,8332.0,5.0,M,2.0,1.0
4,ODB6BA6ORHL2UYZE,4,0.0,744.0,3.0,M,2.0,1.0


In [693]:
identity.drop('GENDER_CODE', axis=1, inplace=True)

In [707]:
most = {'AGE':3, 'CHILDREN_CNT':0, 'EDU_CODE':3, 'INCOME_RANGE_CODE':1, 'WORK_MTHS':1, 'CUST_START_DT':6260}

In [695]:
all_cust = pd.DataFrame(page['CUST_NO'].drop_duplicates())

In [696]:
all_identity = pd.merge(all_cust, identity, how='left')

In [708]:
all_identity.fillna(value=most, inplace=True)

In [799]:
# 編輯資料
all_identity['CUST_START_DT'] = all_identity['CUST_START_DT'].apply(lambda x:x-9447)

In [800]:
all_identity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195000 entries, 0 to 194999
Data columns (total 7 columns):
CUST_NO              195000 non-null object
AGE                  195000 non-null float64
CHILDREN_CNT         195000 non-null float64
CUST_START_DT        195000 non-null float64
EDU_CODE             195000 non-null float64
INCOME_RANGE_CODE    195000 non-null float64
WORK_MTHS            195000 non-null float64
dtypes: float64(6), object(1)
memory usage: 11.9+ MB


In [801]:
all_identity.head()

,CUST_NO,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,AZTHNWQ_LXMGIMYG,3.0,0.0,-1502.0,3.0,1.0,1.0
1,3PY428CHUQBULFIG,3.0,0.0,-3187.0,3.0,1.0,1.0
2,JVPD1QUJWVLMZU8S,4.0,0.0,-5854.0,4.0,1.0,1.0
3,6STXUMWZRDCGSDDU,2.0,0.0,-1761.0,3.0,1.0,1.0
4,YV7EFLVIAQER2KRU,3.0,0.0,-3187.0,3.0,1.0,1.0


In [802]:
all_identity.describe()

,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,INCOME_RANGE_CODE,WORK_MTHS
count,195000.000000,195000.000000,195000.000000,195000.000000,195000.000000,195000.000000
mean,2.817585,0.008918,-3204.079949,3.198513,1.088041,1.017974
std,0.718306,0.136279,1464.713285,0.747509,0.364337,0.134965
min,1.000000,0.000000,-9447.000000,1.000000,1.000000,1.000000
25%,3.000000,0.000000,-3187.000000,3.000000,1.000000,1.000000
50%,3.000000,0.000000,-3187.000000,3.000000,1.000000,1.000000
75%,3.000000,0.000000,-3187.000000,3.000000,1.000000,1.000000
max,4.000000,6.000000,120.000000,6.000000,4.000000,5.000000


# functions

In [176]:
def selector(df, col, min_date, max_date):
    return df[(df[col]>=min_date) & (df[col]<=max_date)]
def cmm(d, key, variable):
    d = d[[key, variable]].groupby(key)
    d_m = pd.merge(pd.merge(d.count(), d.min(), left_index=True, right_index=True), 
               d.max(), left_index=True, right_index=True)
    d_m.rename(columns = {'{}_x'.format(variable): '{}_Count'.format(variable), 
                          '{}_y'.format(variable): '{}_Min'.format(variable), 
                          '{}'.format(variable): '{}_Max'.format(variable)}, inplace = True)
    return d_m.reset_index()
def c(x):
    return x.split(',')
def merge_path(d):
    path_m = d[['CUST_NO', 'new_path']].groupby('CUST_NO')['new_path'].apply(lambda x: "%s" % ','.join(x))
    df_path_m = pd.DataFrame(path_m)
    df_path_m['new_path'] = df_path_m['new_path'].apply(c)
    return df_path_m
# MultiLabelBinarizer
def encoding(d, unique_cust):
    from sklearn.preprocessing import MultiLabelBinarizer
    one_hot = MultiLabelBinarizer()
    path_encod = one_hot.fit_transform(d['new_path'])
    c_path = pd.concat([unique_cust, pd.DataFrame(path_encod, columns=one_hot.classes_)], axis=1)
    return c_path

In [177]:
def f1_score_1(str_report):
    return float(list(filter(lambda x:x!='', str_report.split('\n')[3].split(' ')))[3])
def decicionTest(tra_X, tra_Y, tes_X, tes_Y):
    dtree = DecisionTreeClassifier()
    dtree.fit(tra_X,tra_Y)
    predictions = dtree.predict(tes_X)
    return predictions, classification_report(tes_Y,predictions), confusion_matrix(tes_Y,predictions)
def max_score_day(m_score):
    max_score = max(m_score.values())
    max_days = [i for i in m_score.keys() if m_score[i] == max(m_score.values())]
    avg_max_days = sum(max_days)/len(max_days)
    return max_score, avg_max_days

# N x txn x page

In [46]:
class M(object):
#     交易從第f天開始，持續30天，瀏覽從第f-m天開始，持續m天期間
    def __init__(self, first_txn_day, m_size=30, first_view_day=None):
        self.first_txn_day = first_txn_day
        self.m_size = m_size
        self.first_view_day = first_txn_day - m_size
#     ＋m天後的交易
    def t_p(self):
        txn_ = selector(txn, 'NEW_TXN_DT', self.first_txn_day, (self.first_txn_day+29))
        page_ = selector(d_page, 'NEW_VISITDATE', self.first_view_day, (self.first_txn_day-1))
        txn_page = pd.merge(page_, txn_, on='CUST_NO', how='left')[['CUST_NO', 
                                                                    'new_path', 'NEW_VISITDATE', 'NEW_TXN_DT']]
        return txn_page
#     產生屬性
    def generate(self):
        self.txn_page = self.t_p()
        self.unique_page = cmm(self.txn_page,'CUST_NO', 'NEW_VISITDATE')
        self.unique_txn = cmm(self.txn_page,'CUST_NO', 'NEW_TXN_DT')
        self.merge_path = merge_path(self.txn_page)
        self.unique_cust = self.txn_page['CUST_NO'].drop_duplicates().reset_index().drop('index', axis=1)
        self.c_path = encoding(self.merge_path, self.unique_cust)
        self.all = pd.merge(pd.merge(self.c_path, self.unique_page), self.unique_txn)
        self.all['y'] = self.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
        self.all_x = self.all[list(self.all.columns[1:-4])]
        self.all_y = self.all['y']
    def check(self):
        words = '''first_view_day: {}, m_size: {}, first_txn_day: {}'''
        return words.format(self.first_view_day, self.m_size, self.first_txn_day)

In [184]:
t_31 = M(31, 5)
t_31.generate()
t_31.check()

'first_view_day: 26, m_size: 5, first_txn_day: 31'

In [186]:
t_61 = M(61,5)
t_61.generate()
t_61.check()

'first_view_day: 56, m_size: 5, first_txn_day: 61'

In [187]:
intersect = list(set(list(t_31.all_x.columns)) & set(list(t_61.all_x.columns)))
t_31.all_x = t_31.all_x[intersect]
t_61.all_x = t_61.all_x[intersect]

In [188]:
dtree = DecisionTreeClassifier()
dtree.fit(t_31.all_x,t_31.all_y)
predictions = dtree.predict(t_61.all_x)

In [192]:
print(f1_score_1(classification_report(t_61.all_y,predictions)))

0.16


# train / test

In [44]:
def comparing_score(train_start_day, test_start_day, m_size):
    train = M(train_start_day, m_size)
    train.generate()
    test = M(test_start_day, m_size)
    test.generate()
#     info = 'train - {}\ntest - {}'.format(train.check(), test.check())
    intersect = list(set(list(train.all_x.columns)) & set(list(test.all_x.columns)))
    train.all_x = train.all_x[intersect]
    test.all_x = test.all_x[intersect]
    predictions, report, matrix = decicionTest(train.all_x, train.all_y, test.all_x, test.all_y)
    score = f1_score_1(report)
    return score, [train, test, predictions, report, matrix]

In [47]:
f_score, f_ref = comparing_score(31, 61, 14)
print(f_score)

0.15


In [49]:
# different m size for score
m_score = {}
for i in range(5,15):
    m_score[i] = comparing_score(31, 61, i)[1]
    print(i, end=' ')
#     print('m={}, score={}'.format(i, comparing_score(31, 61, i)[1]))

5 6 7 8 9 10 11 12 13 14 

In [88]:
max_score_day(m_score)

(0.17, 5.0)

# multiple train / test

In [271]:
def x_versus_page(p, first_txn_day, x, first_view_day):
    x_ = selector(x, 'NEW_TXN_DT', first_txn_day, (first_txn_day+29))
    page_ = selector(p, 'NEW_VISITDATE', first_view_day, (first_txn_day-1))
    x_page = pd.merge(page_, x_, on='CUST_NO', how='left')[['CUST_NO', 
                                                                'new_path', 'NEW_VISITDATE', 'NEW_TXN_DT']]
    return x_page

In [792]:
class X(object):
    def __init__(self, first_txn_day, m_size=5, first_view_day=None):
        self.first_txn_day = first_txn_day
        self.m_size = m_size
        self.first_view_day = first_txn_day - m_size
#     基本資料
    def attach(self):
        self.p=d_page
        self.t=txn
        self.c=card
        self.f=fx
        self.l=loan
#     產生屬性
    def generate_1(self, x):
        self.x_page = x_versus_page(self.p, self.first_txn_day, x, self.first_view_day)
        self.unique_page = cmm(self.x_page,'CUST_NO', 'NEW_VISITDATE')
        self.unique_txn = cmm(self.x_page,'CUST_NO', 'NEW_TXN_DT')
        self.merge_path = merge_path(self.x_page)
        self.unique_cust = self.x_page['CUST_NO'].drop_duplicates().reset_index().drop('index', axis=1)
        self.c_path = encoding(self.merge_path, self.unique_cust)
        self.all = pd.merge(pd.merge(self.c_path, self.unique_page), self.unique_txn)
        self.all = pd.merge(self.all, all_identity)
    def generate_2(self):
        self.all = self.all.reindex(sorted(self.all.columns), axis=1)
        self.all['y'] = self.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
        self.all_x = self.all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 'y'], axis=1)
#         self.all_x = self.all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 
#                                   'NEW_VISITDATE_Count', 'NEW_VISITDATE_Max', 'NEW_VISITDATE_Min', 'y'], axis=1)
        self.all_y = self.all['y']

In [455]:
def x_comparing_score(train_start_day, test_start_day, m_size, tar):
#     建立物件
    train = X(train_start_day, m_size)
    test = X(test_start_day, m_size)
    train.attach()
    test.attach()
#     選擇目標
    if tar=='t':
        train.generate_1(train.t)
        test.generate_1(test.t)
        train.generate_2()
        test.generate_2()
    elif tar=='c':
        train.generate_1(train.c)
        test.generate_1(test.c)
        train.generate_2()
        test.generate_2()
    elif tar=='f':
        train.generate_1(train.f)
        test.generate_1(test.f)
        train.generate_2()
        test.generate_2()
    elif tar=='l':
        train.generate_1(train.l)
        test.generate_1(test.l)
        train.generate_2()
        test.generate_2()
#     篩選交集
    intersect = list(set(list(train.all_x.columns)) & set(list(test.all_x.columns)))
    train.all_x = train.all_x[intersect]
    test.all_x = test.all_x[intersect]
    predictions, report, matrix = decicionTest(train.all_x, train.all_y, test.all_x, test.all_y)
    score = f1_score_1(report)
    return score, [train, test, predictions, report, matrix] 

In [734]:
score, ref = x_comparing_score(61, 91, 30, 't')

In [735]:
print(score, ref[0].all.shape, ref[1].all.shape)

0.11 (86911, 60) (86867, 60)


In [738]:
score, ref = x_comparing_score(61, 91, 5, 't')

In [742]:
print(score, ref[0].all.shape, ref[1].all.shape)

0.17 (24133, 54) (27401, 56)


# alter time series

In [793]:
def create_train_test(train_start_day, test_start_day, m_size, tar):

#     建立物件
    train = X(train_start_day, m_size) 
    #     combine multiple test
    tests = []
    for i in range(test_start_day,(test_start_day-30),(m_size*(-1))):
        test = X(i,m_size)
        tests.append(test)
        
    train.attach()
    for test in tests:
        test.attach()
        
#     選擇目標
    if tar=='t':
        train.generate_1(train.t)
        for test in tests:
            test.generate_1(test.t)
    elif tar=='c':
        train.generate_1(train.c)
        for test in tests:
            test.generate_1(test.c)
    elif tar=='f':
        train.generate_1(train.f)
        for test in tests:
            test.generate_1(test.f)
    elif tar=='l':
        train.generate_1(train.l)
        for test in tests:
            test.generate_1(test.l)
            
#     篩選交集
    d = [list(i.all.columns) for i in [train]+tests]
    intersect = list(set(d[0]).intersection(*d))
    train.all = train.all[intersect]
    for i in tests:
        i.all = i.all[intersect]
    train.all = train.all.reindex(sorted(train.all.columns), axis=1)
    for i in tests:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)

    train.all['y'] = train.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    train.all_x = train.all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 'y'], axis=1)
#     train.all_x = train.all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 
#                                   'NEW_VISITDATE_Count', 'NEW_VISITDATE_Max', 'NEW_VISITDATE_Min', 'y'], axis=1)
    train.all_y = train.all['y']
    
    test_all = pd.concat([i.all for i in tests]).groupby('CUST_NO').max().reset_index()
    test_all['y'] = test_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    test_all_x = test_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 'y'], axis=1)
#     test_all_x = test_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 
#                                   'NEW_VISITDATE_Count', 'NEW_VISITDATE_Max', 'NEW_VISITDATE_Min', 'y'], axis=1)
    test_all_y = test_all['y'] 

    return train, test_all, test_all_x, test_all_y

In [804]:
def Create_train_test(train_start_day, test_start_day, m_size, tar):

#     建立物件
    #     combine multiple train
    trains = []
    for i in range(train_start_day,(train_start_day-30),(m_size*(-1))):
        train = X(i,m_size)
#         if i==train_start_day or i==(train_start_day-30+m_size):
#             print('tr_first_view_day: {}'.format(i-m_size), 
#                   'X_Period: {}-{}-{}-{}'.format(i-m_size, (i-1), i, (i+29)))
        trains.append(train)
  
    #     combine multiple test
    tests = []
    for i in range(test_start_day,(test_start_day-30),(m_size*(-1))):
        test = X(i,m_size)
#         if i==test_start_day or i==(test_start_day-30+m_size):
#             print('te_first_view_day: {}'.format(i-m_size), 
#                   'X_Period: {}-{}-{}-{}'.format(i-m_size, (i-1), i, (i+29)))
        tests.append(test)
        
    for train in trains:
        train.attach()
    for test in tests:
        test.attach()
        
#     選擇目標
    if tar=='t':
        for train in trains:
            train.generate_1(train.t)
        for test in tests:
            test.generate_1(test.t)
    elif tar=='c':
        for train in trains:
            train.generate_1(train.c)
        for test in tests:
            test.generate_1(test.c)
    elif tar=='f':
        for train in trains:
            train.generate_1(train.f)
        for test in tests:
            test.generate_1(test.f)
    elif tar=='l':
        for train in trains:
            train.generate_1(train.l)
        for test in tests:
            test.generate_1(test.l)
            
#     篩選交集
    d = [list(i.all.columns) for i in trains+tests]
    intersect = list(set(d[0]).intersection(*d))
    
    for i in trains:
        i.all = i.all[intersect]
    for i in tests:
        i.all = i.all[intersect]
        
    for i in trains:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)
    for i in tests:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)

    train_all = pd.concat([i.all for i in trains]).groupby('CUST_NO').max().reset_index()
    train_all['y'] = train_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    train_all_x = train_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 'y'], axis=1)
    #     train_all_x = train_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 
#                                   'NEW_VISITDATE_Count', 'NEW_VISITDATE_Max', 'NEW_VISITDATE_Min', 'y'], axis=1)
    # 編輯資料
    train_all_x['NEW_VISITDATE_Max'] = train_all_x['NEW_VISITDATE_Max'].apply(lambda x:x-train_start_day)
    train_all_x['NEW_VISITDATE_Min'] = train_all_x['NEW_VISITDATE_Min'].apply(lambda x:x-train_start_day)
    
    train_all_y = train_all['y']
    
    test_all = pd.concat([i.all for i in tests]).groupby('CUST_NO').max().reset_index()
    test_all['y'] = test_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    test_all_x = test_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 'y'], axis=1)
#     test_all_x = test_all.drop(['CUST_NO', 'NEW_TXN_DT_Count', 'NEW_TXN_DT_Max', 'NEW_TXN_DT_Min', 
#                                   'NEW_VISITDATE_Count', 'NEW_VISITDATE_Max', 'NEW_VISITDATE_Min', 'y'], axis=1)
    # 編輯資料
    test_all_x['NEW_VISITDATE_Max'] = test_all_x['NEW_VISITDATE_Max'].apply(lambda x:x-test_start_day)
    test_all_x['NEW_VISITDATE_Min'] = test_all_x['NEW_VISITDATE_Min'].apply(lambda x:x-test_start_day)
    
    test_all_y = test_all['y'] 

    return train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y

In [472]:
def experiment_score(train_all_x, train_all_y, test_all_x, test_all_y):
    predictions, report, matrix = decicionTest(train_all_x, train_all_y, test_all_x, test_all_y)
    score = f1_score_1(report)
    return score, [predictions, report, matrix]

# Score

In [476]:
# without identity
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 30, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.11
c  score: 0.04
f  score: 0.41
l  score: 0.02
train: (92858, 55), test: (86911, 54)


In [743]:
# with identity
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 30, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.1
c  score: 0.05
f  score: 0.37
l  score: 0.03
train: (92858, 61), test: (86911, 60)


In [746]:
# without identity
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 5, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.16
c  score: 0.0
f  score: 0.5
l  score: 0.07
train: (22630, 49), test: (24133, 48)


In [748]:
# with identity
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 5, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.15
c  score: 0.04
f  score: 0.47
l  score: 0.07
train: (22630, 55), test: (24133, 54)


In [483]:
# without identity
for i in ['t', 'c', 'f', 'l']:
    train, test_all, test_all_x, test_all_y = create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train.all_x, train.all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

t  score: 0.17
c  score: 0.02
f  score: 0.43
l  score: 0.1
train: (92858, 42), test: (86911, 43)


In [753]:
# with identity
for i in ['t', 'c', 'f', 'l']:
    train, test_all, test_all_x, test_all_y = create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train.all_x, train.all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

t  score: 0.16
c  score: 0.06
f  score: 0.39
l  score: 0.11
train: (86867, 38), test: (86911, 49)


In [484]:
# without identity
for i in ['t', 'c', 'f', 'l']:
    train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

t  score: 0.12
c  score: 0.04
f  score: 0.42
l  score: 0.13
train: (92858, 42), test: (86911, 42)


In [759]:
# with identity
for i in ['t', 'c', 'f', 'l']:
    train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

tr_first_view_day: 26 X_Period: 26-30-31-60
tr_first_view_day: 1 X_Period: 1-5-6-35
te_first_view_day: 56 X_Period: 56-60-61-90
te_first_view_day: 31 X_Period: 31-35-36-65
t  score: 0.14
tr_first_view_day: 26 X_Period: 26-30-31-60
tr_first_view_day: 1 X_Period: 1-5-6-35
te_first_view_day: 56 X_Period: 56-60-61-90
te_first_view_day: 31 X_Period: 31-35-36-65
c  score: 0.08
tr_first_view_day: 26 X_Period: 26-30-31-60
tr_first_view_day: 1 X_Period: 1-5-6-35
te_first_view_day: 56 X_Period: 56-60-61-90
te_first_view_day: 31 X_Period: 31-35-36-65
f  score: 0.38
tr_first_view_day: 26 X_Period: 26-30-31-60
tr_first_view_day: 1 X_Period: 1-5-6-35
te_first_view_day: 56 X_Period: 56-60-61-90
te_first_view_day: 31 X_Period: 31-35-36-65
l  score: 0.11
train: (92858, 48), test: (86911, 48)


In [59]:
# without identity
for i in ['t', 'c', 'f', 'l']:
    score = x_comparing_score(31, 61, 7, i)[0]
    print('cuurent item: {}'.format(i))
    print('score: {}'.format(score))

cuurent item: t
score: 0.14
cuurent item: c
score: 0.04
cuurent item: f
score: 0.47
cuurent item: l
score: 0.07


In [755]:
# with identity
for i in ['t', 'c', 'f', 'l']:
    score = x_comparing_score(31, 61, 7, i)[0]
    print('cuurent item: {}'.format(i))
    print('score: {}'.format(score))

cuurent item: t
score: 0.13
cuurent item: c
score: 0.03
cuurent item: f
score: 0.45
cuurent item: l
score: 0.06


In [805]:
train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(31, 61, 5, 'f')

In [806]:
train_all_x.head()

,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,INCOME_RANGE_CODE,NEW_VISITDATE_Count,NEW_VISITDATE_Max,NEW_VISITDATE_Min,WORK_MTHS,edrn/deoxt,...,gygrt/wgedtm,s/hoxsgqodr,s/hoxsgqodrdppqa,s/pgusordqqodr,s/pgusordqqodrdppqa,s/pgusordqqodrygukikcdtkor,s/qodrdffktkordqikqg,s/qodrioumfowrqodf,s/seqodr,s/wgdqthcdqcxqdtkor
0,3.0,0.0,-3187.0,3.0,1.0,2,-6,-6,1.0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.0,0.0,-3646.0,6.0,1.0,3,-13,-14,1.0,1,...,0,0,0,0,0,0,0,0,0,0
2,3.0,0.0,-3187.0,3.0,1.0,1,-23,-23,1.0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,0.0,-3187.0,3.0,1.0,1,-9,-9,1.0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.0,0.0,-8095.0,3.0,1.0,1,-5,-5,1.0,0,...,0,0,0,0,0,0,0,0,0,0


In [807]:
test_all_x.head()

,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,INCOME_RANGE_CODE,NEW_VISITDATE_Count,NEW_VISITDATE_Max,NEW_VISITDATE_Min,WORK_MTHS,edrn/deoxt,...,gygrt/wgedtm,s/hoxsgqodr,s/hoxsgqodrdppqa,s/pgusordqqodr,s/pgusordqqodrdppqa,s/pgusordqqodrygukikcdtkor,s/qodrdffktkordqikqg,s/qodrioumfowrqodf,s/seqodr,s/wgdqthcdqcxqdtkor
0,3.0,0.0,-3187.0,3.0,1.0,3,-9,-9,1.0,1,...,0,0,0,0,0,0,0,0,0,0
1,3.0,0.0,-3187.0,3.0,1.0,1,-9,-9,1.0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.0,0.0,-3187.0,3.0,1.0,8,-1,-4,1.0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,0.0,-8095.0,3.0,1.0,2,-21,-25,1.0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.0,0.0,-3187.0,3.0,1.0,1,-22,-22,1.0,1,...,0,0,0,0,0,0,0,0,0,0


# score_collection

In [72]:
def score_collection(train_start_day, test_start_day, tar):
    x_score = {}
    print('m_size:', end=' ')
    for m in range(1,15):
        x_score[m] = x_comparing_score(train_start_day=train_start_day, 
                                       test_start_day=test_start_day, m_size=m, tar=tar)[0]
        print(m, end=' ')
    return x_score

In [81]:
def X_record(tar):
    x_record = {}
    for train_start_day in range(61,30,-1):
        print('train_start_day: {}'.format(train_start_day), end='  ')
        x_score = score_collection(train_start_day=train_start_day, test_start_day=91, tar=tar)   
        x_record[train_start_day] = x_score
        print()
    return x_record

In [82]:
def max_record(x_record):
    max_x_record = {}
    for k in list(x_record.keys()):
        max_x_record[k] = max_score_day(x_record[k])
    x_s = [i[0] for i in list(max_x_record.values())]
    x_d = [i[1] for i in list(max_x_record.values())]
    avg_x_s = sum(x_s) / len(x_s)
    avg_x_d = sum(x_d) / len(x_d)
    return avg_x_s, avg_x_d

In [131]:
for i in [t_record, c_record, f_record, l_record]:
    max_i = max_record(i)
    print('max_score: {}, max_date: {}'.format(max_i[0], max_i[1]))

max_score: 0.1770967741935484, max_date: 4.072580645161291
max_score: 0.05516129032258067, max_date: 7.181720430107528
max_score: 0.5003225806451612, max_date: 1.096774193548387
max_score: 0.0964516129032258, max_date: 4.196236559139784


# Test

In [760]:
scs = []
for i in ['t', 'c', 'f', 'l']:
    train, test_all, test_all_x, test_all_y = create_train_test(61, 91, 5, i)
    score, ref = experiment_score(train.all_x, train.all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    scs.append(score)
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))
score_sum = 20*scs[0] + 10*scs[1] + scs[2] + 20*scs[3]
print('simulating score sum: {}'.format(score_sum))

t  score: 0.16
c  score: 0.1
f  score: 0.38
l  score: 0.12
train: (92858, 48), test: (86867, 50)
simulating score sum: 6.98


In [761]:
scs = []
for i in ['t', 'c', 'f', 'l']:
    train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(61, 91, 5, i)
    score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    scs.append(score)
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))
score_sum = 20*scs[0] + 10*scs[1] + scs[2] + 20*scs[3]
print('simulating score sum: {}'.format(score_sum))

tr_first_view_day: 56 X_Period: 56-60-61-90
tr_first_view_day: 31 X_Period: 31-35-36-65
te_first_view_day: 86 X_Period: 86-90-91-120
te_first_view_day: 61 X_Period: 61-65-66-95
t  score: 0.13
tr_first_view_day: 56 X_Period: 56-60-61-90
tr_first_view_day: 31 X_Period: 31-35-36-65
te_first_view_day: 86 X_Period: 86-90-91-120
te_first_view_day: 61 X_Period: 61-65-66-95
c  score: 0.1
tr_first_view_day: 56 X_Period: 56-60-61-90
tr_first_view_day: 31 X_Period: 31-35-36-65
te_first_view_day: 86 X_Period: 86-90-91-120
te_first_view_day: 61 X_Period: 61-65-66-95
f  score: 0.4
tr_first_view_day: 56 X_Period: 56-60-61-90
tr_first_view_day: 31 X_Period: 31-35-36-65
te_first_view_day: 86 X_Period: 86-90-91-120
te_first_view_day: 61 X_Period: 61-65-66-95
l  score: 0.17
train: (86911, 47), test: (86867, 47)
simulating score sum: 7.4


In [783]:
def test_range(a, b):
    for t in [1,2,3,5,6]:
        print('t={}'.format(t))
        scs = []
        for i in ['t', 'c', 'f', 'l']:
            train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(a, b, t, i)
            score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
            print('{}  score: {}'.format(i, score))
            scs.append(score)
            if i=='l':
                print('train: {}, test: {}'.format(train_all.shape, test_all.shape))
        score_sum = 20*scs[0] + 10*scs[1] + scs[2] + 20*scs[3]
        print('simulating score sum: {}'.format(score_sum))
    print('\n', end='   ')

In [784]:
d = [(31, 61), (41, 71), (51, 81), (61, 91)]
for p in d:
    print('test_range: {}'.format(p))
    test_range(p[0], p[1])

test_range: (31, 61)
t=1
t  score: 0.17
c  score: 0.08
f  score: 0.41
l  score: 0.14
train: (92858, 33), test: (86911, 33)
simulating score sum: 7.41
t=2


KeyboardInterrupt: 

In [767]:
for t in [1,2,3,5,6,10,15,30]:
    print('t={}'.format(t))
    scs = []
    for i in ['t', 'c', 'f', 'l']:
        train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(31, 61, t, i)
        score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
        print('{}  score: {}'.format(i, score))
        scs.append(score)
        if i=='l':
            print('train: {}, test: {}'.format(train_all.shape, test_all.shape))
    score_sum = 20*scs[0] + 10*scs[1] + scs[2] + 20*scs[3]
    print('simulating score sum: {}'.format(score_sum))

t=1
t  score: 0.17
c  score: 0.08
f  score: 0.42
l  score: 0.13
train: (92858, 33), test: (86911, 33)
simulating score sum: 7.220000000000001
t=2
t  score: 0.15
c  score: 0.07
f  score: 0.4
l  score: 0.15
train: (92858, 39), test: (86911, 39)
simulating score sum: 7.1000000000000005
t=3
t  score: 0.16
c  score: 0.07
f  score: 0.39
l  score: 0.13
train: (92858, 42), test: (86911, 42)
simulating score sum: 6.890000000000001
t=5
t  score: 0.15
c  score: 0.08
f  score: 0.38
l  score: 0.11
train: (92858, 48), test: (86911, 48)
simulating score sum: 6.38
t=6
t  score: 0.13
c  score: 0.07
f  score: 0.39
l  score: 0.08
train: (92858, 49), test: (86911, 49)
simulating score sum: 5.290000000000001
t=10
t  score: 0.12
c  score: 0.07
f  score: 0.36
l  score: 0.11
train: (92858, 52), test: (86911, 52)
simulating score sum: 5.66
t=15
t  score: 0.14
c  score: 0.06
f  score: 0.37
l  score: 0.07
train: (92858, 54), test: (86911, 54)
simulating score sum: 5.170000000000001
t=30
t  score: 0.1
c  score: 0

In [765]:
for t in [1,2,3,5,6,10,15,30]:
    print('t={}'.format(t))
    scs = []
    for i in ['t', 'c', 'f', 'l']:
        train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(61, 91, t, i)
        score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
        print('{}  score: {}'.format(i, score))
        scs.append(score)
        if i=='l':
            print('train: {}, test: {}'.format(train_all.shape, test_all.shape))
    score_sum = 20*scs[0] + 10*scs[1] + scs[2] + 20*scs[3]
    print('simulating score sum: {}'.format(score_sum))

t=1
t  score: 0.17
c  score: 0.11
f  score: 0.42
l  score: 0.12
train: (86911, 33), test: (86867, 33)
simulating score sum: 7.32
t=2
t  score: 0.15
c  score: 0.12
f  score: 0.42
l  score: 0.18
train: (86911, 39), test: (86867, 39)
simulating score sum: 8.219999999999999
t=3
t  score: 0.15
c  score: 0.11
f  score: 0.41
l  score: 0.17
train: (86911, 41), test: (86867, 41)
simulating score sum: 7.91
t=5
t  score: 0.13
c  score: 0.1
f  score: 0.4
l  score: 0.17
train: (86911, 47), test: (86867, 47)
simulating score sum: 7.4
t=6
t  score: 0.13
c  score: 0.1
f  score: 0.41
l  score: 0.15
train: (86911, 48), test: (86867, 48)
simulating score sum: 7.01
t=10
t  score: 0.14
c  score: 0.1
f  score: 0.38
l  score: 0.11
train: (86911, 51), test: (86867, 51)
simulating score sum: 6.380000000000001
t=15
t  score: 0.13
c  score: 0.08
f  score: 0.35
l  score: 0.08
train: (86911, 52), test: (86867, 52)
simulating score sum: 5.3500000000000005
t=30
t  score: 0.11
c  score: 0.05
f  score: 0.28
l  score: 

# combine

In [542]:
zero = pd.read_csv('Downloads/dataset/TBN_Y_ZERO.csv')
zero.shape

(30000, 5)

In [618]:
# test
tests = {}

# train
trains = {}

# predict
prds = {}

# print('cuurent item:', end=' ')
for i in ['t', 'c', 'f', 'l']:
    print('cuurent item: {}'.format(i))
    train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(91, 121, 5, i)
    score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
    trains[i] = train_all
    tests[i] = test_all
    prds[i] = ref[0]

cuurent item: t
tr_first_view_day: 86 X_Period: 86-90-91-120
tr_first_view_day: 81 X_Period: 81-85-86-115
tr_first_view_day: 76 X_Period: 76-80-81-110
tr_first_view_day: 71 X_Period: 71-75-76-105
tr_first_view_day: 66 X_Period: 66-70-71-100
tr_first_view_day: 61 X_Period: 61-65-66-95
te_first_view_day: 116 X_Period: 116-120-121-150
te_first_view_day: 111 X_Period: 111-115-116-145
te_first_view_day: 106 X_Period: 106-110-111-140
te_first_view_day: 101 X_Period: 101-105-106-135
te_first_view_day: 96 X_Period: 96-100-101-130
te_first_view_day: 91 X_Period: 91-95-96-125
cuurent item: c
tr_first_view_day: 86 X_Period: 86-90-91-120
tr_first_view_day: 81 X_Period: 81-85-86-115
tr_first_view_day: 76 X_Period: 76-80-81-110
tr_first_view_day: 71 X_Period: 71-75-76-105
tr_first_view_day: 66 X_Period: 66-70-71-100
tr_first_view_day: 61 X_Period: 61-65-66-95
te_first_view_day: 116 X_Period: 116-120-121-150
te_first_view_day: 111 X_Period: 111-115-116-145
te_first_view_day: 106 X_Period: 106-110-111

In [621]:
trains['c'].head()

,CUST_NO,NEW_TXN_DT_Count,NEW_TXN_DT_Max,NEW_TXN_DT_Min,NEW_VISITDATE_Count,NEW_VISITDATE_Max,NEW_VISITDATE_Min,edrn/deoxt,edrn/fkjktdq,edrn/guuou.dspz,...,s/hoxsgqodr,s/hoxsgqodrdppqa,s/pgusordqqodr,s/pgusordqqodrdppqa,s/pgusordqqodrygukikcdtkor,s/qodrdffktkordqikqg,s/qodrioumfowrqodf,s/seqodr,s/wgdqthcdqcxqdtkor,y
0,---CHVW7DUN8SZLO,0,NaN,NaN,2,86,86,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,--1YNJZSSOXDF6UU,0,NaN,NaN,3,86,86,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,--3NF5SUEEEAWOPI,0,NaN,NaN,2,88,87,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,--8BVNAAVAUSW6X8,0,NaN,NaN,1,71,71,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,--9QPGEBDTE8XEWM,0,NaN,NaN,2,86,86,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [622]:
# merge test & predict
custPrds = {}
for i in ['t', 'c', 'f', 'l']:
    tests[i]['{}_prediction'.format(i)] = prds[i]
    custPrds[i] = tests[i][['CUST_NO', '{}_prediction'.format(i)]]

In [625]:
for i in ['t', 'c', 'f', 'l']:
    print(custPrds[i].shape, end=' ')
    print(custPrds[i][custPrds[i]['{}_prediction'.format(i)]==1].shape, end=' ')
    print('\n')

(30000, 2) (227, 2) 

(30000, 2) (513, 2) 

(30000, 2) (1547, 2) 

(30000, 2) (234, 2) 



In [606]:
# 檢查t預測為1的名單
check_list = list(custPrds['t'][custPrds['t']['{}_prediction'.format('t')]==1]['CUST_NO'].head())

In [608]:
check_list

['0BPVXUZOO02JK7Q8',
 '0KVRLSFQXQMFMXKK',
 '0XDRT9MACNYSMOEC',
 '0XMY-DCGBVZ20ZPO',
 '1E4KWWWWI7Y-R5CK']

In [599]:
from functools import reduce
d = [zero, custPrds['c'], custPrds['f'], custPrds['l'], custPrds['t']]
result = reduce(lambda left,right: pd.merge(left,right,how='left',on='CUST_NO'), d)

In [609]:
# 檢查t預測值為1的名單，是否合併後預測值為1
result[result['CUST_NO'].isin(check_list)]

,CUST_NO,CC_IND,FX_IND,LN_IND,WM_IND,c_prediction,f_prediction,l_prediction,t_prediction
742,0XDRT9MACNYSMOEC,0,0,0,0,0,1,0,1
19376,0BPVXUZOO02JK7Q8,0,0,0,0,0,0,0,1
19620,1E4KWWWWI7Y-R5CK,0,0,0,0,0,1,0,1
20502,0XMY-DCGBVZ20ZPO,0,0,0,0,1,1,0,1
23447,0KVRLSFQXQMFMXKK,0,0,0,0,0,0,0,1


In [610]:
result.drop(['CC_IND', 'FX_IND', 'LN_IND', 'WM_IND'], axis=1, inplace=True)
result.rename(columns = {'c_prediction': 'CC_IND', 
                         'f_prediction': 'FX_IND', 
                         'l_prediction': 'LN_IND', 
                         't_prediction': 'WM_IND', }, inplace = True)

In [611]:
result.head()

,CUST_NO,CC_IND,FX_IND,LN_IND,WM_IND
0,_PT5HFBEZJKOZ934,0,0,0,0
1,6STXUMWZRDCGSDDU,0,0,0,0
2,JDVF4U8JUANEID68,0,0,0,0
3,8I6SQDGP9OQYUN1M,0,0,0,1
4,R-TRDUV3GHTID31I,0,0,0,0


In [617]:
result.to_csv('result_0303.csv', index=True)

In [810]:
zero.columns

Index(['CUST_NO', 'CC_IND', 'FX_IND', 'LN_IND', 'WM_IND'], dtype='object')

In [811]:
zero_new = zero.copy()

In [813]:
for i in ['CC_IND', 'FX_IND', 'LN_IND', 'WM_IND']:
    zero_new[i] = 1

In [815]:
zero_new.set_index("CUST_NO", inplace=True)

In [816]:
zero_new.head()

,CC_IND,FX_IND,LN_IND,WM_IND
CUST_NO,,,,
_PT5HFBEZJKOZ934,1,1,1,1
6STXUMWZRDCGSDDU,1,1,1,1
JDVF4U8JUANEID68,1,1,1,1
8I6SQDGP9OQYUN1M,1,1,1,1
R-TRDUV3GHTID31I,1,1,1,1


In [817]:
zero_new.to_csv('zero_new.csv', index=True)